##### A. To create main dataset with all the required features for patient similarity trajectory plot 
*****
Script goal: To create main dataset for patient similarity trajectory plot

The script does the following:
1. The dataset is created by merging the original dataset (`Full_ICD10_ATC`) with the computed cluster labels (`Cluster3Label_Full_ICD10_ATC_Dummies_ICD10_ATC_20`) after clustering.
2. Merge the original, cluster labels dataset, use the age dataset (`age_dataset.csv`) to get the actual age in floating age of the patient
3. Save the dataset with the name `trajectory_df.csv`, with 12 sub dataset based on age group, gender and presence/absence of ADHD in patient's diagnostic 
history that contains all required columns for the patient similarity trajectory plot
*****

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

# Load environment variables from .env file
load_dotenv()

# Read the dataset with cluster labels, original dataset & actual age of patient dataset
cluster_df = pd.read_csv(os.getenv("cluster_df_path"))
original_df = pd.read_csv(os.getenv("orginal_df_path"))
age_dataset = pd.read_csv(os.getenv("age_dataset_path"))

# Merge the original dataset with the cluster labels, and age dataset
trajectory_df = original_df[
    [
        "pasient",
        "episode_id",
        "episode_start_date",
        "episode_end_date",
        "gender",
        "age_group",
        "diagnosis",
        "actual_med_Full_ATC",
        "Length_of_Episode",
        "Count_visit",
        "Therapy_ratio",
        "tillnextepisode",
    ]
].merge(
    cluster_df[["episode_id", "cluster"]],
    on="episode_id",
    how="inner",
)

trajectory_df = trajectory_df.merge(
    age_dataset[["episode_id", "fdt"]],
    on="episode_id",
    how="inner",
)
print(trajectory_df["age_group"].value_counts())
# Get age of patient by substracting episode_start_date from fdt and round it to 2 decimal places
trajectory_df["episode_start_date"] = pd.to_datetime(
    trajectory_df["episode_start_date"]
)
trajectory_df["fdt"] = pd.to_datetime(trajectory_df["fdt"])
trajectory_df["age"] = (
    trajectory_df["episode_start_date"] - trajectory_df["fdt"]
).dt.days
trajectory_df["age"] = round(trajectory_df["age"] / 365.2425, 2)

# Drop the fdt columns
trajectory_df.drop("fdt", axis=1, inplace=True)

pasient_min = (
    trajectory_df.groupby("pasient")
    .agg(
        {
            "age": "min",
        }
    )
    .reset_index()
)
# rename the column name from age to minimum_age for pasient_min dataframe
pasient_min.rename(columns={"age": "initial_age"}, inplace=True)

# Now use the pasient_min dataframe to merge with the trajectory_df dataframe, add a new column minimum_age in the trajectory_df dataframe.
# Use the pasient column to merge the two dataframes, if they have same pasient id then add the corresponding age from pasient_min to the trajectory_df minimum_age column for all matched pasient id.
trajectory_df = trajectory_df.merge(
    pasient_min[["pasient", "initial_age"]],
    on="pasient",
    how="inner",
)


# From trajectory_df use the minimum_age column and if it is between 0-5 then assign age_group as Preschooler (include 0 to 5.9),
# is minimum_age between 6-11 then age_group is MiddleChildhood (include 0 to 5.9), if minimum_age between 12-18 (include 0 to 5.9) then age_group is Teenager
def assign_age_group(minimum_age):
    if 0 <= minimum_age < 6:
        return "Preschooler"
    elif 6 <= minimum_age < 12:
        return "MiddleChildhood"
    elif 12 <= minimum_age <= 19:
        return "Teenager"


trajectory_df["initial_age_group"] = trajectory_df["initial_age"].apply(
    assign_age_group
)


display(trajectory_df.head(60))
print(trajectory_df.shape)
print(trajectory_df.columns)


# Save trajectory_df dataset to csv by selecting only the columns
trajectory_df[
    [
        "pasient",
        "episode_id",
        "episode_start_date",
        "episode_end_date",
        "gender",
        "age",
        "initial_age",
        "diagnosis",
        "actual_med_Full_ATC",
        "Length_of_Episode",
        "Count_visit",
        "Therapy_ratio",
        "tillnextepisode",
        "cluster",
        "age_group",
        "initial_age_group",
    ]
].to_csv(
    os.getenv("trajectory_df_path"),
    index=False,
)

##### B. Create main dataset with a column ADHD,  
1. If any of the episodes diagnosis of the patient contains `F90` encode it as `1`, 
2. If it contains all `NaN` in all the episodes of patient encode it as `2` , 
3. If it does not contain any `F90`, neither it has `NaN` in all the episodes, then encode it as `0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("trajectory_df_path"))


# Function to determine the encoding for ADHD
def encode_adhd(diagnoses):
    # Check if any diagnosis contains 'F90'
    if any("F90" in str(d) for d in diagnoses):
        return 1
    # Other than, return 0
    else:
        return 0


"""
    if all(pd.isna(d) for d in diagnoses):
        return 2
    # Check if any diagnosis contains 'F90'
    elif any("F90" in str(d) for d in diagnoses):
        return 1
    # If neither of the above, return 3
    else:
        return 0
"""

# Group by 'pasient' and apply the encoding function to the 'diagnosis' column
trajectory_df["ADHD"] = trajectory_df.groupby("pasient")["diagnosis"].transform(
    encode_adhd
)

# if trajectory_df["ADHD"] == 2 then replace it with 0
trajectory_df["ADHD"] = trajectory_df["ADHD"].replace(2, 0)

# Count total unique patients with 3 in ADHD & save the dataset
print(f"Total Patients: {trajectory_df['pasient'].nunique()}")
print(
    f"Patients with ADHD: {trajectory_df[trajectory_df['ADHD'] == 1]['pasient'].nunique()}"
)
print(
    f"Patients without ADHD or NaN: {trajectory_df[trajectory_df['ADHD'] == 0]['pasient'].nunique()}"
)
display(trajectory_df.head(60))
trajectory_df.to_csv(
    os.getenv("updated_trajectory_df_path"),
    index=False,
)

# Verify the distribution of patients and episodes dataset
print("\n")
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))
print(f"On new Main Dataset, Total Episodes: {trajectory_df['episode_id'].nunique()}")

print(
    f"On new Main Dataset Total Number of Episodes: {trajectory_df['episode_id'].nunique()}"
)
print(
    f"On new Main Dataset Total Number of Pasients: {trajectory_df['pasient'].nunique()}"
)

print(
    f"Gender Distribution (As gender 0 is excluded in the study):{trajectory_df['gender'].value_counts()}"
)

len(trajectory_df[trajectory_df["age"] > 18])
# Load and view the formed distribution of ADHD after above encoding
# 0       10436
# 2        6495
# 1        5745

##### C. All the possible combinations for patient similarity are based on `AgeGroup = ["Preschooler", "MiddleSchool", "Teenager"]` || `Gender = ["M", "F"]` || `ADHD = [0, 1]`, they are :

1. Preschooler, Male, ADHD: (Preschooler, M, 1)

2. Preschooler, Male, No ADHD: (Preschooler, M, 0)

3. Preschooler, Female, ADHD: (Preschooler, F, 1)

4. Preschooler, Female, No ADHD: (Preschooler, F, 0)

5. MiddleChildhood, Male, ADHD: (MiddleChildhood, M, 1)

6. MiddleChildhood, Male, No ADHD: (MiddleChildhood, M, 0)

7. MiddleChildhood, Female, ADHD: (MiddleChildhood, F, 1)

8. MiddleChildhood, Female, No ADHD: (MiddleChildhood, F, 0)

9. Teenager, Male, ADHD: (Teenager, M, 1)

10. Teenager, Male, No ADHD: (Teenager, M, 0)

11. Teenager, Female, ADHD: (Teenager, F, 1)

12. Teenager, Female, No ADHD: (Teenager, F, 0)

##### 1. Create a dataset containing only those rows which satisfy this condition `Gender = M`, `AgeGroup = PreSchooler` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))
print(f"Total Patients: {trajectory_df['pasient'].nunique()}")
# Count total number of patients with ADHD as 1
count_df = trajectory_df[trajectory_df["ADHD"] == 1]
# Select only those data from trajectory_df which has Gender as M, AgeGroup as PreSchooler and ADHD as 1 and save it in a new dataframe preSchooler_M_ADHD
preSchooler_M_ADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "Preschooler")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {preSchooler_M_ADHD['pasient'].nunique()}")
print(f"Episode: {preSchooler_M_ADHD.shape}")
display(preSchooler_M_ADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_PreSchooler_M_ADHD.csv"
)
preSchooler_M_ADHD.to_csv(subset_dataset, index=False)

##### 2. Create a dataset containing only those rows which satify this condition `Age = M`, `AgeGroup = PreSchooler` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as M, AgeGroup as PreSchooler and ADHD as 0 and save it in a new dataframe preSchooler_M_NoADHD
preSchooler_M_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "Preschooler")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {preSchooler_M_NoADHD['pasient'].nunique()}")
print(f"Episode: {preSchooler_M_NoADHD.shape}")
display(preSchooler_M_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_PreSchooler_M_NoADHD.csv"
)
preSchooler_M_NoADHD.to_csv(subset_dataset, index=False)

##### 3. Create a dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = PreSchooler` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as PreSchooler and ADHD as 1 and save it in a new dataframe preSchooler_F_ADHD
preSchooler_F_ADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "Preschooler")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {preSchooler_F_ADHD['pasient'].nunique()}")
print(f"Episode: {preSchooler_F_ADHD.shape}")
display(preSchooler_F_ADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_PreSchooler_F_ADHD.csv"
)
preSchooler_F_ADHD.to_csv(subset_dataset, index=False)

##### 4. Create a Dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = PreSchooler` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as PreSchooler and ADHD as 0 and save it in a new dataframe preSchooler_F_NoADHD
preSchooler_F_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "Preschooler")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {preSchooler_F_NoADHD['pasient'].nunique()}")
print(f"Episode: {preSchooler_F_NoADHD.shape}")
display(preSchooler_F_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_PreSchooler_F_NoADHD.csv"
)
preSchooler_F_NoADHD.to_csv(subset_dataset, index=False)

##### 5. Create a dataset containing only those rows which satify this condition `Gender = M`, `AgeGroup = MiddleChildhood` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as M, AgeGroup as MiddleChildhood and ADHD as 1 and save it in a new dataframe MiddleChildhood_M_ADHD
MiddleChildhood_M_ADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "MiddleChildhood")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {MiddleChildhood_M_ADHD['pasient'].nunique()}")
print(f"Episode: {MiddleChildhood_M_ADHD.shape}")
display(MiddleChildhood_M_ADHD.head(5))

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_MiddleChildhood_M_ADHD.csv"
)
MiddleChildhood_M_ADHD.to_csv(subset_dataset, index=False)

#### 6. Create a dataset containing only those rows which satify this condition `Gender = M`, `AgeGroup = MiddleChildhood` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as M, AgeGroup as MiddleChildhood and ADHD as 0 and save it in a new dataframe MiddleChildhood_M_NoADHD
MiddleChildhood_M_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "MiddleChildhood")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {MiddleChildhood_M_NoADHD['pasient'].nunique()}")
print(f"Episode: {MiddleChildhood_M_NoADHD.shape}")
display(MiddleChildhood_M_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_MiddleChildhood_M_NoADHD.csv"
)
MiddleChildhood_M_NoADHD.to_csv(subset_dataset, index=False)

##### 7. Create a dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = MiddleChildhood` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as MiddleChildhood and ADHD as 1 and save it in a new dataframe MiddleChildhood_F_ADHD
MiddleChildhood_F_ADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "MiddleChildhood")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {MiddleChildhood_F_ADHD['pasient'].nunique()}")
print(f"Episode: {MiddleChildhood_F_ADHD.shape}")
display(MiddleChildhood_F_ADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_MiddleChildhood_F_ADHD.csv"
)
MiddleChildhood_F_ADHD.to_csv(subset_dataset, index=False)

##### 8. Create a dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = MiddleChildhood` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as MiddleChildhood and ADHD as 0 and save it in a new dataframe MiddleChildhood_F_NoADHD
MiddleChildhood_F_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "MiddleChildhood")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {MiddleChildhood_F_NoADHD['pasient'].nunique()}")
print(f"Episode: {MiddleChildhood_F_NoADHD.shape}")
display(MiddleChildhood_F_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(
    subset_dataset_folder, "SubDataset_MiddleChildhood_F_NoADHD.csv"
)
MiddleChildhood_F_NoADHD.to_csv(subset_dataset, index=False)

##### 9. Create a dataset containing only those rows which satify this condition `Gender = M`, `AgeGroup = Teenager` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as M, AgeGroup as Teenager and ADHD as 1 and save it in a new dataframe Teenager_M_ADHD
Teenager_M_ADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "Teenager")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {Teenager_M_ADHD['pasient'].nunique()}")
print(f"Episode: {Teenager_M_ADHD.shape}")
display(Teenager_M_ADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(subset_dataset_folder, "SubDataset_Teenager_M_ADHD.csv")
Teenager_M_ADHD.to_csv(subset_dataset, index=False)

##### 10. Create a dataset containing only those rows which satify this condition `Gender = M`, `AgeGroup = Teenager` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as M, AgeGroup as Teenager and ADHD as 1 and save it in a new dataframe Teenager_M_NoADHD
Teenager_M_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "M")
    & (trajectory_df["initial_age_group"] == "Teenager")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {Teenager_M_NoADHD['pasient'].nunique()}")
print(f"Episode: {Teenager_M_NoADHD.shape}")
display(Teenager_M_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(subset_dataset_folder, "SubDataset_Teenager_M_NoADHD.csv")
Teenager_M_NoADHD.to_csv(subset_dataset, index=False)

##### 11. Create a dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = Teenager` and `ADHD = 1`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as Teenager and ADHD as 1 and save it in a new dataframe Teenager_F_ADHD
Teenager_F_ADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "Teenager")
    & (trajectory_df["ADHD"] == 1)
]
print(f"Patients: {Teenager_F_ADHD['pasient'].nunique()}")
print(f"Episode: {Teenager_F_ADHD.shape}")
display(Teenager_F_ADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(subset_dataset_folder, "SubDataset_Teenager_F_ADHD.csv")
Teenager_F_ADHD.to_csv(subset_dataset, index=False)

##### 12. Create a dataset containing only those rows which satify this condition `Gender = F`, `AgeGroup = Teenager` and `ADHD = 0`

In [ ]:
from allpackages import warnings, ast, os, pd, load_dotenv

load_dotenv()
trajectory_df = pd.read_csv(os.getenv("updated_trajectory_df_path"))

# Select only those data from trajectory_df which has Gender as F, AgeGroup as Teenager and ADHD as 0 and save it in a new dataframe Teenager_F_NoADHD
Teenager_F_NoADHD = trajectory_df[
    (trajectory_df["gender"] == "F")
    & (trajectory_df["initial_age_group"] == "Teenager")
    & (trajectory_df["ADHD"] == 0)
]
print(f"Patients: {Teenager_F_NoADHD['pasient'].nunique()}")
print(f"Episode: {Teenager_F_NoADHD.shape}")
display(Teenager_F_NoADHD.info())

# Get SubsetDataset folder path from environment variables & subset dataset path CSV file
subset_dataset_folder = os.getenv("SubsetDataset")
subset_dataset = os.path.join(subset_dataset_folder, "SubDataset_Teenager_F_NoADHD.csv")
Teenager_F_NoADHD.to_csv(subset_dataset, index=False)

##### D. Description of the obtained reasult
**Episode counts**
1. Total number of records in dataframes of all combinations = 
592+1447+178+1066+2449+3468+899+2311+912+3030+715+5562 = 22629

2. Total number of records in the initial dataframe = 22629 + 47(Gender 0 which is excluded) = 22676

**Patient counts**
1. Total number of patients in dataframes of all combinations = 
363+1211+113+880+1857+2978+641+1873+829+2834+608+5014 = 19201 + 47 = 19248

2. Total number of records in the initial dataframe = .... (Gender 0 which is excluded) 
